# Entrenamiento y predicción con BigQuery ML

## Introducción

Este tutorial muestra cómo entrenar y evaluar un modelo de clasificación binaria con BQML. También usaremos este modelo para generar predicciones.

Las dos primeras partes del tutorial explican cómo extraer los datos y prepararlos antes de entrenar su modelo.

La última parte del tutorial profundiza en el código de entrenamiento que se usa para este modelo, con un enfoque particular en los requisitos para hacerlo compatible con BigQuery ML.

## Conjunto de Datos

Este tutorial utiliza el conjunto de datos de ingresos del censo de los Estados Unidos proporcionado por el repositorio de aprendizaje automático de UC Irvine. Este conjunto de datos contiene información sobre personas de una base de datos del Censo de 1994, incluida la edad, la educación, el estado civil, la ocupación y si ganan más de $50,000 al año.

El conjunto de datos ahora existe en los conjuntos de datos públicos de BigQuery:

`bigquery-public-data.ml_datasets.census_adult_income`